### AB Testing

AB testing course by Google on Udacity

https://towardsdatascience.com/the-math-behind-a-b-testing-with-example-code-part-1-of-2-7be752e1d06f

https://www.kaggle.com/tammyrotem/ab-tests-with-python

### AB testing - ecommerce exercise

Undertanding the results of and AB test run by an e-commerce website.
Analyze the data to help the company decide if they should implement the new page, keep the old one, or run the experiment longer to make the decision.

Data from: https://www.kaggle.com/zhangluyuan/ab-testing?select=ab_data.csv

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(42)

In [36]:
import statsmodels.api as sm
from scipy import stats

In [2]:
data = pd.read_csv('AB testing - data.csv')
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


user_id - The user ID of each session

timestamp - Timestamp for the session

group - Which group the user was assigned to for that session {control, treatment}

landing_page - Which design each user saw on that session {old_page, new_page}

converted - Whether the session ended in a conversion or not (binary, 0=not converted, 1=converted)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [4]:
data.converted.unique()

array([0, 1])

In [5]:
data.landing_page.unique()

array(['old_page', 'new_page'], dtype=object)

In [6]:
data.group.unique()

array(['control', 'treatment'], dtype=object)

In [7]:
data.user_id.nunique()

290584

In [8]:
data.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [9]:
#cheking proportion of users converted
data.query('converted == 1').user_id.count()/data.user_id.count()

0.11965919355605512

For the rows where 'treatment' is not aligned with 'new_page' or 'control' is not aligned with 'old_page', we can not be sure if this row truly recieved the new or old page. We should remove this rows.

In [10]:
#line ups between landing_page and group
print(data.query('landing_page == "old_page" and group == "treatment"').user_id.count())

print(data.query('landing_page == "new_page" and group == "control"').user_id.count())


1965
1928


In [11]:
#delete the rows we can not be confident in the accuracy of its data
d_a = data.query('landing_page == "old_page" and group == "control"')
d_b = data.query('landing_page == "new_page" and group == "treatment"')
data_ab = d_a.append(d_b, ignore_index = True)
data_ab

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,864975,2017-01-21 01:52:26.210827,control,old_page,1
3,936923,2017-01-10 15:20:49.083499,control,old_page,0
4,719014,2017-01-17 01:48:29.539573,control,old_page,0
...,...,...,...,...,...
290580,677163,2017-01-03 19:41:51.902148,treatment,new_page,0
290581,925675,2017-01-07 20:38:26.346410,treatment,new_page,0
290582,643562,2017-01-02 19:20:05.460595,treatment,new_page,0
290583,822004,2017-01-04 03:36:46.071379,treatment,new_page,0


In [12]:
data_ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290585 entries, 0 to 290584
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290585 non-null  int64 
 1   timestamp     290585 non-null  object
 2   group         290585 non-null  object
 3   landing_page  290585 non-null  object
 4   converted     290585 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.1+ MB


In [13]:
#double check all the necesary  rows were deleted
data_ab[((data_ab['group'] == 'treatment') == (data_ab['landing_page'] == 'new_page')) == False].shape[0]

0

In [14]:
#double check no users are repeated 
data_ab.user_id.nunique()

290584

In [15]:
data_ab[data_ab.user_id.duplicated()]

,user_id,timestamp,group,landing_page,converted
146678,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [16]:
data_ab.iloc[146678]

user_id                             773192
timestamp       2017-01-14 02:55:59.590927
group                            treatment
landing_page                      new_page
converted                                0
Name: 146678, dtype: object

In [18]:
data_ab = data_ab.drop(146678, axis = 0)
data_ab = data_ab.reset_index(drop = True)

In [19]:
#check it dropped correctly
data_ab.iloc[146678]

user_id                             884145
timestamp       2017-01-09 15:45:38.457508
group                            treatment
landing_page                      new_page
converted                                0
Name: 146678, dtype: object

###### What is the probability of a user recieving the new page?

In [21]:
data_ab.query('landing_page == "new_page"').user_id.count()/data_ab.user_id.count()

0.5000619442226688

###### Given that a user was in the 'treatment' group, what is the probability they converted?

In [22]:
data_ab.query('group == "treatment"').converted.mean()

0.11880806551510564

###### Given that a user was in the 'control' group, what is the probability they converted?

In [23]:
data_ab.query('group == "control"').converted.mean()

0.1203863045004612

###### What is the probability of a user converting regardless of the page given?

In [24]:
data_ab.converted.mean()

0.11959708724499628

Given the closeness of the results of this probabilities, I would say we don't have enough evidence to say if the new page leads to more conversions. 

#### A regression approach

Use statsmodels to fit the regression model to see if there is a significant difference in conversion based on which page a user is shown.

In [35]:
data_ab['intercept'] = 1

data_ab[['new_page', 'old_page']] = pd.get_dummies(data_ab['landing_page'])

data_ab['ab_page'] = pd.get_dummies(data_ab['group'])['treatment']

data_ab

,user_id,timestamp,group,landing_page,converted,intercept,new_page,old_page,ab_page
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,1,0,1,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,1,0,1,0
2,864975,2017-01-21 01:52:26.210827,control,old_page,1,1,0,1,0
3,936923,2017-01-10 15:20:49.083499,control,old_page,0,1,0,1,0
4,719014,2017-01-17 01:48:29.539573,control,old_page,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
290579,677163,2017-01-03 19:41:51.902148,treatment,new_page,0,1,1,0,1
290580,925675,2017-01-07 20:38:26.346410,treatment,new_page,0,1,1,0,1
290581,643562,2017-01-02 19:20:05.460595,treatment,new_page,0,1,1,0,1
290582,822004,2017-01-04 03:36:46.071379,treatment,new_page,0,1,1,0,1


In [38]:
log_model = sm.Logit(data_ab['converted'], data_ab[['intercept','ab_page']])
result = log_model.fit()

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


In [39]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290582
Method:                           MLE   Df Model:                            1
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:               8.077e-06
Time:                        16:58:18   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1899
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9888      0.008   -246.669      0.000      -2.005      -1.973
ab_page       -0.0150      0.011     -1.311      0.190      -0.037       0.007
==============================================================================
"""

P-Value is 0.19 which means 'ab_page' is not that significant in predicting whether or not the users convert. 

##### Where to go from here:


-Could analyze the timestamp column to add a layer of understanding to the success/failure of the new model.

-Could add an effect based on which country a user lives in. Does it appear that country had an impact on conversion?

In [42]:
countries = pd.read_csv('AB testing - countries.csv')
data_new = countries.set_index('user_id').join(data_ab.set_index('user_id'),
                                               how = 'inner')
data_new

,country,timestamp,group,landing_page,converted,intercept,new_page,old_page,ab_page
user_id,,,,,,,,,
834778,UK,2017-01-14 23:08:43.304998,control,old_page,0,1,0,1,0
928468,US,2017-01-23 14:44:16.387854,treatment,new_page,0,1,1,0,1
822059,UK,2017-01-16 14:04:14.719771,treatment,new_page,1,1,1,0,1
711597,UK,2017-01-22 03:14:24.763511,control,old_page,0,1,0,1,0
710616,UK,2017-01-16 13:14:44.000513,treatment,new_page,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...
653118,US,2017-01-09 03:12:31.034796,control,old_page,0,1,0,1,0
878226,UK,2017-01-05 15:02:50.334962,control,old_page,0,1,0,1,0
799368,UK,2017-01-09 18:07:34.253935,control,old_page,0,1,0,1,0


In [43]:
data_new.country.unique()

array(['UK', 'US', 'CA'], dtype=object)

In [44]:
data_new[['UK', 'US', 'CA']] = pd.get_dummies(data_new['country'])
data_new

,country,timestamp,group,landing_page,converted,intercept,new_page,old_page,ab_page,UK,US,CA
user_id,,,,,,,,,,,,
834778,UK,2017-01-14 23:08:43.304998,control,old_page,0,1,0,1,0,0,1,0
928468,US,2017-01-23 14:44:16.387854,treatment,new_page,0,1,1,0,1,0,0,1
822059,UK,2017-01-16 14:04:14.719771,treatment,new_page,1,1,1,0,1,0,1,0
711597,UK,2017-01-22 03:14:24.763511,control,old_page,0,1,0,1,0,0,1,0
710616,UK,2017-01-16 13:14:44.000513,treatment,new_page,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
653118,US,2017-01-09 03:12:31.034796,control,old_page,0,1,0,1,0,0,0,1
878226,UK,2017-01-05 15:02:50.334962,control,old_page,0,1,0,1,0,0,1,0
799368,UK,2017-01-09 18:07:34.253935,control,old_page,0,1,0,1,0,0,1,0


In [45]:
log_model_2 = sm.Logit(data_new['converted'], data_new[['intercept','ab_page','US','UK']])
result_countries = log_model_2.fit()

Optimization terminated successfully.
         Current function value: 0.366113
         Iterations 6


In [46]:
result_countries.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290580
Method:                           MLE   Df Model:                            3
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:               2.323e-05
Time:                        18:35:33   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1760
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9893      0.009   -223.763      0.000      -2.007      -1.972
ab_page       -0.0149      0.011     -1.307      0.191      -0.037       0.007
US             0.0099      0.013      0.743      0.457      -0.016       0.036
UK            -0.0408      0.027     -1.516      0.130      -0.093       0.012
==============================================================================
"""

###### Check now the interaction between page and country to see if there are any significant effects on conversion.

In [47]:
log_model_us = sm.Logit(data_new['converted'], data_new[['intercept','ab_page','US']])
result_US = log_model_us.fit()

Optimization terminated successfully.
         Current function value: 0.366117
         Iterations 6


In [49]:
result_US.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290581
Method:                           MLE   Df Model:                            2
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:               1.233e-05
Time:                        18:38:36   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.2693
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9919      0.009   -228.420      0.000      -2.009      -1.975
ab_page       -0.0150      0.011     -1.309      0.191      -0.037       0.007
US             0.0126      0.013      0.952      0.341      -0.013       0.038
==============================================================================
"""

In [51]:
log_model_uk = sm.Logit(data_new['converted'], data_new[['intercept','ab_page','UK']])
result_UK = log_model_us.fit()

Optimization terminated successfully.
         Current function value: 0.366117
         Iterations 6


In [52]:
result_UK.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290581
Method:                           MLE   Df Model:                            2
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:               1.233e-05
Time:                        18:39:44   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.2693
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9919      0.009   -228.420      0.000      -2.009      -1.975
ab_page       -0.0150      0.011     -1.309      0.191      -0.037       0.007
US             0.0126      0.013      0.952      0.341      -0.013       0.038
==============================================================================
"""